### Backtester

Below is a detailed backtester function to be used to backtest each spread trading strategy 

In [1]:
import pandas as pd
import numpy as np

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~FUNCTION: Backtester ~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Inputs:
#     - a optimal entry threshold
#     - mu long termmean of the OU process to adjust optimal entry
#     - order - the sreadto be tested
#     -oil_data - dataframe containg relevant data 

def backtester(a, mu, order, oil_data):
    bt_df = pd.DataFrame()
    #Calculate log spread
    bt_df['spread'] = np.log((oil_data[order[0]]/oil_data[order[1]]))
    
    # calculate log returns of each asset 
    bt_df['A_returns'] = np.log(oil_data[order[0]]/oil_data[order[0]].shift(1))
    bt_df['B_returns'] = np.log(oil_data[order[1]]/oil_data[order[1]].shift(1))
    
    #Calc optimal entry/exitthresholds
    entry = a + mu
    exit = -1*a + mu
    
    # create long_short signals 
    # long trade when spread < entry
    # short trade when spread > exit
    bt_df['Long_short'] = 0
    bt_df['Long_short'] = np.where(bt_df.spread < entry, 1, bt_df.Long_short)
    bt_df['Long_short'] = np.where(bt_df.spread > exit, -1, bt_df.Long_short)
    
    # Fill neutral signals with previous signal
    bt_df['Long_short_fill'] =  bt_df['Long_short'].replace(to_replace=0, method='ffill')
    
    # Calculate dail profit and loss for active spread trading
    bt_df['pnl'] = bt_df.Long_short_fill * bt_df.B_returns - bt_df.Long_short_fill * bt_df.A_returns
    
    # Calculate total return
    total_return = bt_df.pnl.sum()
    
    # Calcualte sharpe ratio
    sharpe = bt_df.pnl.mean() / bt_df.pnl.std() * np.sqrt(252)
    
    #calculate mean daily return 
    mean_return = bt_df.pnl.mean()
    
    # Calculate trade count
    trade_count = 0 
    for i in range(1,len(bt_df)):
        if bt_df.loc[i, 'Long_short_fill'] != bt_df.loc[i-1, 'Long_short_fill']:
            trade_count += 1
    # calculate average hold time        
    if trade_count != 0:
        avg_hold_time = len(bt_df) / trade_count
    else:
        avg_hold_time = 0 
    return total_return, mean_return, sharpe, trade_count, avg_hold_time